In [19]:
from sklearn.datasets import fetch_20newsgroups
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np


In [20]:
trainData = pd.read_csv("LIAR2/train.csv", delimiter=',')
trainData['statement'] = trainData['statement'].astype(str)
statements = trainData['statement'].tolist()

display(trainData.head())

,id,label,statement,date,subject,speaker,speaker_description,state_info,true_counts,mostly_true_counts,half_true_counts,mostly_false_counts,false_counts,pants_on_fire_counts,context,justification
0,13847,5,"90 percent of Americans ""support universal bac...","October 2, 2017",government regulation;polls and public opinion...,chris abele,"Chris Abele is Milwaukee County Executive, a p...",wisconsin,1,4,5,3,5,2,a tweet,"""Universal"" is the term for background checks ..."
1,13411,1,Last year was one of the deadliest years ever ...,"May 19, 2017",after the fact;congress;criminal justice;histo...,thom tillis,Thom Tillis is a Republican who serves as U.S....,north carolina,0,2,7,3,2,0,a press release supporting the Back The Blue A...,"Sen. Thom Tillis, a North Carolina Republican,..."
2,10882,0,"Bernie Sanders's plan is ""to raise your taxes ...","October 28, 2015",taxes,chris christie,"Chris Christie announced June 6, 2023 that he ...",national,21,20,27,11,17,8,"Boulder, Colo","Christie said that Sanders’s plan is ""to raise..."
3,20697,4,Voter ID is supported by an overwhelming major...,"December 8, 2021",voter id laws,lee zeldin,Lee Zeldin is a Republican representing New Yo...,new york,1,2,0,0,0,0,a Tweet,Zeldin claimed voter identification requiremen...
4,6095,2,"Says Barack Obama ""robbed Medicare (of) $716 b...","August 12, 2012",federal budget;history;medicare;retirement,mitt romney,Mitt Romney is a U.S. senator from Utah. He ra...,national,31,33,58,35,32,19,"an interview on ""60 Minutes""","Romney said, ""There's only one president that ..."


In [21]:
## No finetuning used
# Initialize BERTopic model
topic_model = BERTopic()
# Fit the model on the statements
topics, probs = topic_model.fit_transform(statements)

# To view the topics and their frequencies
topic_model.get_topic_info()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

,Topic,Count,Name,Representation,Representative_Docs
0,-1,8362,-1_the_to_says_and,"[the, to, says, and, of, for, in, that, is, on]","[Says Trump told Fox News ""the Democrats can s..."
1,0,807,0_school_schools_education_college,"[school, schools, education, college, students...",[Says that despite a recent increase in school...
2,1,407,1_abortion_parenthood_planned_abortions,"[abortion, parenthood, planned, abortions, wom...","[Says Wisconsin Gov. Tony Evers ""wants to forc..."
3,2,239,2_ballots_election_fraud_mail,"[ballots, election, fraud, mail, voter, arizon...","[Ballots in Arizona's Maricopa County"" from th..."
4,3,219,3_biden_joe_video_shows,"[biden, joe, video, shows, bidens, president, ...","[Video shows Joe Biden has a body double., Say..."
...,...,...,...,...,...
237,236,10,236_deficit_trillion_decade_took,"[deficit, trillion, decade, took, oneyear, map...",[Deficit spending 'exploded during the Obama a...
238,237,10,237_georgias_georgia_investments_film,"[georgias, georgia, investments, film, industr...",[The Georgia Department of Economic Developme...
239,238,10,238_rainy_fund_day_226,"[rainy, fund, day, 226, 643, lawmakers, 201213...","[Thanks to our policies, ""for the first time i..."
240,239,10,239_davis_wendy_ayotte_cheese,"[davis, wendy, ayotte, cheese, socialists, kel...","[Says Wendy Davis, ""born into difficult circum..."


In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load data
data = pd.read_csv("LIAR2/train.csv", delimiter=',')

# Filter necessary columns and drop missing values
data = data[['statement', 'subject']].dropna()

# Encode 'subject' as numerical labels
label_encoder = LabelEncoder()
data['subject'] = label_encoder.fit_transform(data['subject'])

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data['statement'], data['subject'], test_size=0.2, random_state=42)


In [23]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

# Initialize BERTopic model with BERT embeddings
topic_model = BERTopic(embedding_model="paraphrase-MiniLM-L6-v2")

# Fit the model on training data statements
topics, probabilities = topic_model.fit_transform(X_train)


In [24]:
# Retrieve topics and their representations
topic_info = topic_model.get_topic_info()
display(topic_info.head(100))

# Map each topic to the actual subject label, if a pattern exists
# Alternatively, use embeddings to train a classifier (next step)


,Topic,Count,Name,Representation,Representative_Docs
0,-1,7016,-1_the_to_of_and,"[the, to, of, and, in, for, says, that, is, has]",[House Speaker Nancy Pelosi said this week tha...
1,0,691,0_school_schools_education_scott,"[school, schools, education, scott, students, ...",[Teachers in Chicago Public Schools are paid s...
2,1,362,1_covid_19_vaccine_vaccines,"[covid, 19, vaccine, vaccines, vaccinated, cdc...","[White House, CDC, WHO and vaccine makers don'..."
3,2,325,2_biden_joe_shows_video,"[biden, joe, shows, video, president, hunter, ...",[Document shows that Hunter Biden paid Joe Bid...
4,3,312,3_ballots_election_voter_voting,"[ballots, election, voter, voting, mail, vote,...","[November 2012 was ""a record year of turnout. ..."
...,...,...,...,...,...
95,94,19,94_jersey_new_job_christie,"[jersey, new, job, christie, jobs, growth, cre...",[When it comes to being effective at creating ...
96,95,19,95_trafficking_human_camps_central,"[trafficking, human, camps, central, task, cag...","[Because of the Trans-Pacific Partnership ""we'..."
97,96,18,96_china_korea_north_communists,"[china, korea, north, communists, elections, w...","[It's an absolute fact"" that China and North K..."
98,97,18,97_returns_released_tax_release,"[returns, released, tax, release, return, romn...",[Mitt Romney is the first major party candidat...


import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from bertopic import BERTopic

# Load your data
data = pd.read_csv("LIAR2/train.csv")

# Extract the statements and the subject
X = data['statement']  # Text data
y = data['subject']  # Labels (topics)

# Check for missing values in X (input) and y (target)
print("Missing values in X (input):", X.isnull().sum())
print("Missing values in y (target):", y.isnull().sum())

# Remove rows where either X or y has NaN values
data = data.dropna(subset=['statement', 'subject'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['statement'], data['subject'], test_size=0.2, random_state=42)

# Initialize BERTopic model
topic_model = BERTopic()

# Fit the model on the training data
topic_model.fit(X_train.tolist())

# Get embeddings using the transform method for the training set
train_embeddings, _ = topic_model.transform(X_train.tolist())

# Ensure the embeddings are a 2D array
train_embeddings = np.array(train_embeddings)

# Check for missing values in the embeddings and handle them
if np.any(np.isnan(train_embeddings)):
    print("Missing values detected in train_embeddings")
    # Option 1: Fill NaNs with zero (useful for embeddings with missing features)
    train_embeddings = np.nan_to_num(train_embeddings, nan=0.0)

# Ensure embeddings are 2D
if len(train_embeddings.shape) != 2:
    print("Train embeddings are not 2D, reshaping to 2D.")
    train_embeddings = train_embeddings.reshape(len(train_embeddings), -1)  # Reshaping to 2D

# Train the Logistic Regression classifier on the embeddings
classifier = LogisticRegression(max_iter=1000)
classifier.fit(train_embeddings, y_train)

# Get embeddings for the test set using the transform method
test_embeddings, _ = topic_model.transform(X_test.tolist())

# Ensure the test embeddings are a 2D array
test_embeddings = np.array(test_embeddings)

# Check for missing values in the embeddings and handle them
if np.any(np.isnan(test_embeddings)):
    print("Missing values detected in test_embeddings")
    # Option 1: Fill NaNs with zero
    test_embeddings = np.nan_to_num(test_embeddings, nan=0.0)

# Ensure embeddings are 2D
if len(test_embeddings.shape) != 2:
    print("Test embeddings are not 2D, reshaping to 2D.")
    test_embeddings = test_embeddings.reshape(len(test_embeddings), -1)  # Reshaping to 2D

# Predict on the test set
y_pred = classifier.predict(test_embeddings)

# Evaluate the model using accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
